# Load Stuff

In [1]:
""" DEBUG """
%cd /root/Documents    
%run "Source Code/Setup Notebook.ipynb" #<-- Debug
%run "Source Code/Setup Source.ipynb"   #<-- Debug

/root/Documents


# Limiting "Observe" calls 

we limit the number of time that "observe" is called per second for jupyter functions

In [ ]:
def size(b):
    if time.time() - Graph.w.time > .01:  #<-- make function run only ever 1/10 a second
        Graph.w.time = time.time()
        values = "['"+str(Graph.w.size.value)+"','"+str(Graph.w.size.value)+"']" #<-- values for changing node's style
        display(Javascript("""
        for( var i=0; i < cy.io.selection.length; i++ ){
            var ele = cy.io.selection[i]
            if( ele.isNode() ){
                cy.io.style.change( ele, ["width","height"], """+values+"""   )   
            }else{
                cy.io.style.change( ele, "width", '"""+str(Graph.w.size.value/6)+"""'   )   
            }
        }
        """))           
Graph.wf.size = size
Graph.w.size.observe( Graph.wf.size )

#  <font color="teal">Style</font> <font color="orange">Functions</font> 
The following code sets up the <font color="orange">javascript</font> functions/variables that handle the style nodes and edges
Create object that holds style stuff. Add to it the default style of nodes and edges

## <font color="orange">Old Set Style</font>

In [ ]:
JS("""
cy.io.style.set = function(ele, self=false){
    var style = (ele.isNode() ) ? cy.io.style.node : cy.io.style.edge ;
    
    if( self == false ){ //<-- set properties of element
        for (var property in style ) {
            if( style[property] != "NaNpx" && !property.includes("pie") ){   // removes error messages from weird stuff in cytoscape.js
                ele.style( property, style[property] )
           }
        }    
    }

};
""")

# Helper: <font color="magenta">JS Box</font>

This box was created to solve the a bunch of the glitches with the "running javascript commands" **<font color="red">Explain More</font>**

In [ ]:
jsBox = JupyterCell( "Javascript Box", "js_num" )
def run(text):
    jsBox.text = text
    jsBox.execute()
jsBox.run = run

In [ ]:
Javascript("alert('howdy ho')")

<font color="red">For debugging</font>

In [ ]:
# Javascript Box
JS(jsBox.text) 
clear_output()

# Helper: <font color="magenta">Output</font><font color="orange">.JS</font> <font color="red">unfinished</font>

**<font color="magenta">OutputJS</font>** is a very useful <font color="magenta">python class</font>/<font color="orange">javascript function</font> that allows variable to go from: <font color="orange">javascript</font> $\rightarrow$ <font color="blue">python</font>.

**Python Code**
```python
JS("OutputJS( 'howdy ho!' )")
OutputJS.var
```
** Output**
```
'howdy ho!'
```

In [ ]:
class Output():
    var = ""
    JS("""
    function sleep(ms){ 
        return new Promise( resolve => setTimeout(resolve, ms)); 
    }; 
    """)
    JS("""
    async function OutputJS(text){ 
        Jupyter.notebook.kernel.execute( 'Output.var = "'+String(text)+'"' );  //<-- write text to python variable: Output.var 
        count = 1
        while( Jupyter.notebook.kernel_busy && count < 25  ){                  //<-- wait for variable to be written, then continue
            console.log( Jupyter.notebook.kernel_busy  )
            console.log( count )
            count += 1
            await sleep(100);
        }
        console.log("one more sleep")
        console.log( Jupyter.notebook.kernel_busy )
        await sleep(1000);
    };  
    """)
    
    def JS(text):
        JS("""OutputJS("""+text+""")""")
        time.sleep(.1)
        return Output.var
        

In [ ]:
Output.var

In [ ]:
JS("OutputJS(x)")
time.sleep(10)
print(var)

In [ ]:
Output.var

## TRY 2

In [ ]:
js_sent = False
js_object = {}
count = 0

In [ ]:
def commFunc(comm, msg):
    @comm.on_msg
    def _recv(msg):
        global js_sent; global js_object
        js_object = msg["content"]["data"]
        js_sent = True
        # Use msg['content']['data'] for the data in the message

get_ipython().kernel.comm_manager.register_target('js_comm', commFunc)

JS("comm = Jupyter.notebook.kernel.comm_manager.new_comm('js_comm',{'foo': 6})")

In [ ]:
aio_pool = ThreadPoolExecutor(1)
aio_loop = asyncio.new_event_loop()
aio_pool.submit(asyncio.set_event_loop, aio_loop).result()

async def myFunc():
    global js_sent; global count
    count = 0
    while( (not js_sent)  and count<20 ):
        time.sleep(.1)
        count += 1

async def waiter( js_object ):
    global js_sent
    js_sent = False
    JS("comm.send("+js_object+")")    
    await commFunc
    return
     

def getJS( text ):
    global js_object
    aio_pool.submit( aio_loop.run_until_complete, waiter(text)  ).result()
    return js_object

## Try 3

In [ ]:
def commFunc(comm, msg):
    @comm.on_msg
    def _recv(msg):
        global js_sent; global js_object
        js_object = msg["content"]["data"]
        js_sent = True
        # Use msg['content']['data'] for the data in the message

get_ipython().kernel.comm_manager.register_target('js_comm', commFunc)
JS("comm = Jupyter.notebook.kernel.comm_manager.new_comm('js_comm',{'foo': 6})")

In [ ]:
import asyncio
from concurrent.futures import ThreadPoolExecutor
import IPython

In [ ]:
aio_pool = ThreadPoolExecutor(1)
aio_loop = asyncio.new_event_loop()
aio_pool.submit(asyncio.set_event_loop, aio_loop).result()

async def myFunc():
    global js_sent; global count
    count = 0
    while( (not js_sent)  and count<20 ):
        time.sleep(.1)
        count += 1

async def waiter( js_object ):
    global js_sent
    js_sent = False
    JS("comm.send("+js_object+")")    
    await commFunc
    return
     

def getJS( text ):
    global js_object
    aio_pool.submit( aio_loop.run_until_complete, waiter(text)  ).result()
    return js_object

In [ ]:
JS("comm.send({'foo':1})") 

In [ ]:
x = getJS("{'foo':6}")
x

In [ ]:
count

In [ ]:
js_sent

In [ ]:
comm.

In [ ]:
x= 0
print(x)
JS("Jupyter.notebook.kernel.execute('x=20 ')")
x=1
print(x)
time.sleep(2)
print(x)

## ASYNCIO

[example](https://github.com/ipython/ipython/issues/11030) and a [tutorial](https://stackabuse.com/python-async-await-tutorial/)


In [ ]:
import asyncio
from concurrent.futures import ThreadPoolExecutor

aio_pool = ThreadPoolExecutor(1)
aio_loop = asyncio.new_event_loop()
aio_pool.submit(asyncio.set_event_loop, aio_loop).result()

async def mycoro():
    await asyncio.sleep(1)
    return 5

result = aio_pool.submit(aio_loop.run_until_complete, mycoro()).result()

## COMMS

[tutorial](http://jupyter-notebook.readthedocs.io/en/latest/comms.html)

## Part 1

In [20]:
JS("Jupyter.notebook.kernel.comm_manager.register_target('my_comm_target', function(comm, msg) {});")

In [21]:
from ipykernel.comm import Comm
global x

# Use comm to send a message from the kernel
my_comm = Comm(target_name='my_comm_target')
my_comm.send({'foo': 2})

# Add a callback for received messages.
@my_comm.on_msg
def _recv(msg):
    x = "ja"
    # Use msg['content']['data'] for the data in the message

In [22]:
my_comm.send({'foo': 2})

In [15]:

lines = inspect.getsource(my_comm.open)
print(lines)

    def open(self, data=None, metadata=None, buffers=None):
        """Open the frontend-side version of this comm"""
        if data is None:
            data = self._open_data
        comm_manager = getattr(self.kernel, 'comm_manager', None)
        if comm_manager is None:
            raise RuntimeError("Comms cannot be opened without a kernel "
                        "and a comm_manager attached to that kernel.")

        comm_manager.register_comm(self)
        try:
            self._publish_msg('comm_open',
                              data=data, metadata=metadata, buffers=buffers,
                              target_name=self.target_name,
                              target_module=self.target_module,
                              )
            self._closed = False
        except:
            comm_manager.unregister_comm(self)
            raise



<bound method Comm.open of <ipykernel.comm.comm.Comm object at 0x7f5ac4633be0>>

## Part 2

In [ ]:
my_comm.comm_id

In [ ]:
my_comm.send({'foo': 2})

In [ ]:
x["content"]["data"]

In [ ]:
comm.send({'foo': 5})

In [ ]:
display("hello")

In [ ]:
get_ipython().asdf

## Scratch Paper

In [72]:
x
def target_func(comm, msg):
    # comm is the kernel Comm instance
    # msg is the comm_open message

    # Register handler for later messages
    @comm.on_msg
    def _recv(msg):
        global x
        x = ['content']['data']
        # Use msg['content']['data'] for the data in the message

    # Send data to the frontend
    comm.send({'foo': 5})

get_ipython().kernel.comm_manager.register_target('my_comm_target', target_func)

In [33]:

checkit = get_ipython().get_ipython().kernel.comm_manager


In [37]:
checkit.notify_change

<bound method HasTraits.notify_change of <ipykernel.comm.manager.CommManager object at 0x7f5ac46e59b0>>

In [43]:
checkit2 = checkit.comms['1fdfd3ee6877412e8c61df98e467c0aa']

In [71]:
checkit2.trait_metadata()

TypeError: trait_metadata() missing 2 required positional arguments: 'traitname' and 'key'

In [ ]:
comm = Jupyter.notebook.kernel.comm_manager.new_comm('my_comm_target', {'foo': 6})
// Send data
comm.send({'foo': 7})

// Register a handler
comm.on_msg(function(msg) {
    console.log(msg.content.data.foo);
});

In [48]:
lines = inspect.getsource(checkit2.kernel.instance)
print(lines)

    @classmethod
    def instance(cls, *args, **kwargs):
        """Returns a global instance of this class.

        This method create a new instance if none have previously been created
        and returns a previously created instance is one already exists.

        The arguments and keyword arguments passed to this method are passed
        on to the :meth:`__init__` method of the class upon instantiation.

        Examples
        --------

        Create a singleton class using instance, and retrieve it::

            >>> from traitlets.config.configurable import SingletonConfigurable
            >>> class Foo(SingletonConfigurable): pass
            >>> foo = Foo.instance()
            >>> foo == Foo.instance()
            True

        Create a subclass that is retrived using the base class instance::

            >>> class Bar(SingletonConfigurable): pass
            >>> class Bam(Bar): pass
            >>> bam = Bam.instance()
            >>> bam == Bar.instance()
          

In [26]:
def target_func(comm, msg):
    # comm is the kernel Comm instance
    # msg is the comm_open message

    # Register handler for later messages
    @comm.on_msg
    def _recv(msg):
        # Use msg['content']['data'] for the data in the message

    # Send data to the frontend
    comm.send({'foo': 5})

get_ipython().kernel.comm_manager.register_target('my_comm_target', target_func)

IndentationError: expected an indented block (<ipython-input-26-2552cd3c2387>, line 11)

# Old Widgets

## <font color="green">Control Panel</font> 

**<font color="green">widget : </font>**

In [ ]:
Graph.w.ControlPanel= widgets.ToggleButtons(
    options=[ 'Edit','Search','Graph Style'])
#Graph.w.ControlPanel

**<font color="blue">Function : </font>**  <font color="blue">ControlChanged</font> changes the editorBox display when the Control Panel changes.

In [ ]:
def ControlChanged(b):
    #widg = Graph.w.ControlPanel.value.replace(" ", "")
    #Graph.editor_box.widg = getattr(Graph.w, widg).show
    #Graph.editor_box.execute()
    #clear_output()
    #display(Graph.w.ControlPanel)
    pass

    
Graph.w.ControlPanel.observe( ControlChanged )

How to change button style

```python
Graph.w.ControlPanel.value = "Search"
```

## <font color="green">All nodes</font>

In [ ]:
Graph.w.Edit.allNodes = ipw.Button(description='All Nodes')

In [ ]:
def allNodes(b):
    JS("cy.nodes().select()")   #<-- delete elements
Graph.wf.allNodes = allNodes
Graph.w.Edit.allNodes.on_click( Graph.wf.allNodes )

## <font color="green">All edges</font>

In [ ]:
Graph.w.Edit.allEdges = ipw.Button(description='All Edges')

In [ ]:
def allEdges(b):
    JS("cy.edges().select()")   #<-- delete elements
Graph.wf.allEdges = allEdges
Graph.w.Edit.allEdges.on_click( Graph.wf.allEdges )

## <font color="green">Add Node</font>

In [ ]:
Graph.w.AddNode = type("widget",(),{})()

In [ ]:
Graph.w.AddNode.head = ipw.HTML("""<h4>Describe the New Node, then Click <font color="blue">Add</font></h4>""")
Graph.w.AddNode.id = ipw.Text(description='ID :')
Graph.w.AddNode.button = ipw.Button(description='Add', button_style='info')
Graph.w.AddNode.show = ipw.VBox( [Graph.w.AddNode.head, Graph.w.AddNode.id, Graph.w.AddNode.button ] )
Graph.w.AddNode.show

### <font color="blue">addNode</font>

<font color="blue">addNode</font> adds a nodes to the network

**<font color="red">Function NOT Implemented</font>**

In [ ]:
def AddNode(b):
    w_id= Graph.w.AddNode.id.value
    JS("""
    cy.add([
      { group: "nodes", data: { id: '{w_id}' } , position: { x: 100, y: 200 } },
    ]);
    """)
Graph.wf.AddNode = AddNode
Graph.w.AddNode.button.on_click( Graph.wf.AddNode )  #<-- function when widget is clicked

## <font color="green">Add Edge</font>

In [ ]:
Graph.w.AddEdge = type("widget",(),{})()
Graph.w.AddEdge.show = ipw.HTML("""<h4 class="text-danger">Widget NOT Implemented</h4>""")

### <font color="blue">addEdge</font>

<font color="blue">addEdge</font> adds an edge to the network

**<font color="red">Function NOT Implemented</font>**

In [ ]:
def AddEdge(b):#self, source, target, id):
    """JS(
    cy.add([
      { group: "edges", data: { id: "e0", source: "n0", target: "n1" } }
    ]);
    )"""
Graph.wf.AddEdge = AddEdge

# Previous Default Style

This style was what was previously called inside of the **Graph.start** command

In [ ]:
    style: [ { selector: 'node', style: { 'height': 20, 'width': 20, 'background-color': 'gray' } },
     { selector: 'edge', style: { 'width': 5, 'opacity': 0.5, 'line-color': 'gray'} } ]

#  JS commands

## <font color="orange">selection Highligh</font>


This functions makes the nodes change color when they are selected

In [ ]:
def selectColor(self):
    JS("""
    cy.on("select", 'node' , function(){
        this.style({'background-blacken': .4 });
    });
    cy.on("unselect", 'node' , function(){
        this.style({'background-blacken': 0 });
    });
    """)
JS_Helper.selectColor = selectColor

## Editor_Box changer (tried to stop flashes)

In [ ]:
JS("""
    var check = 0;
    var check_var;
    cy.io.last_type = "Nothing";

    cy.io.test = function(event){
        cy.io.selection = cy.$(':selected')                 //<-- Get selected elements
        cy.io.last_type = cy.io.select_type 
         
        if( cy.io.selection.length == 0  ){                 //<-- Do nothing if nothing's selected
            return
        } else if( cy.io.selection.length == 1 ){           //<-- If selection lenght is 1
        
            if( cy.io.selection.isNode() && cy.io.last_type != "1 node" ){                      //<-- If it's a node
                console.log("selection changed")
                cy.io.select_type = "1 node"
                Jupyter.notebook.kernel.execute("Graph.editor_box.widg = Graph.w.Edit.boxAll")
                IPython.notebook.execute_cells([cy.io.edit_num])  //<-- cy.io.edit_num was setup by the Graph.editor_box
                
            } else if(  cy.io.last_type != "1 edge" ){                                              //<-- If it's an edge
                cy.io.select_type = "1 edge"
                Jupyter.notebook.kernel.execute("Graph.editor_box.widg = Graph.w.Edit.boxAll")
                IPython.notebook.execute_cells([cy.io.edit_num])  //<-- cy.io.edit_num was setup by the Graph.editor_box
            }
            
        } else {                                           //<-- If Many elements
            console.log("many items")
        }
        
    };

    cy.on('select unselect', 'node, edge', cy.io.test );
""")

## When Testing Stuff:

TEST: Add nodes

In [ ]:
JS("""
    cy.add([
      { group: "nodes" },
    ]);
    
    cy.layout({ name: 'circle'}).run();
""")

TEST: Get clicked node info to console

In [ ]:
JS("""

cy.nodes().on("click", function(){
  console.log( this )
});
""")

TEST: get more specific node info:

In [ ]:
JS("""

cy.nodes().on("click", function(){
  console.log( this.data() );
  console.log( " " );console.log( " " );
  console.log( this.style );
});
""")

TEST: Change color on click

In [ ]:
JS("""

cy.nodes().on("click", function(){
    this.style('background-color', 'magenta');
});
""")

TEST: Another color change

In [ ]:
JS("""

cy.nodes().on("click", function(){
    this.style({'background-color': 'blue'});
});
""")

TEST: Get node position

In [ ]:
JS("""

cy.nodes().on("click", function(){
    console.log( this.position() )
});
""")

In [ ]:
JS("""

cy.nodes().on("click", function(){
    console.log( this.style() )
});
""")



TEST: Remove Listeners

In [ ]:
JS("""
cy.nodes().removeListener( "click" )
cy.nodes().removeListener( "select" )
cy.removeListener( "tap" )
""")

TEST: Selected Element

In [ ]:
JS("""
console.log( cy.$(':selected') )
""")

TEST: Jquery commands [Things you can search for](http://js.cytoscape.org/#notation/elements-json) ... [Mayber things u search for?](http://js.cytoscape.org/#selectors/state)

In [ ]:
JS("""
console.log( cy.$(':selectable') )
""")

**TEST: Change Something on select [Things you can select](http://js.cytoscape.org/#events/user-input-device-events)**

In [ ]:
JS("""

cy.nodes().removeListener( "select" )

cy.nodes().on("select", function(){
    this.style({'background-blacken': .3 });
});

cy.nodes().on("unselect", function(){
    this.style({'background-blacken': 0 });
});
""")

** TEST: Add nodes when clicking **

In [ ]:
JS("""
    var checkit;
    
    
    cy.removeListener( "tap" )            // <-- Clears all previous cy "tap" evens
    cy.on('tap', function(event){        // <-- on "tapping" canvas
      if( event.target === cy ){         // <-- If we tapped on the background

          checkit = cy.add([ {  group: "nodes", position: event.position  } ]);            

      } 
    });
""")

**TEST: Add Edge on double click**

In [ ]:
JS("""
cy.removeListener( "tap" )  
""")

In [ ]:
JS("""
    cy.io.click_hist = ["1", "2", "3", "4"];
    
    cy.removeListener( "click" )                               // <-- Clears all previous cy "tap" evens
    cy.on("click" , function(event){                              // <-- on "tapping" canvas
    
      cy.io.click_hist.shift();                                         //<-- Remove oldest click
      
      if( event.target !== cy && event.target.isNode() ){         // <-- If we tapped a Node
          cy.io.click_hist.push( event.target.id() );                  // <-- Add node id to click history
          
          if( cy.io.click_hist[0]==cy.io.click_hist[1] && cy.io.click_hist[2]==cy.io.click_hist[3] ){      //<-- If 2 double clicks
              
              cy.add([ {  data: { source: cy.io.click_hist[0], target: cy.io.click_hist[2] }  } ]); //<-- add edge
          }
          
          
      }else{                                           //<-- if we didn't click a node
          cy.io.click_hist.push( String(Math.random()) );   //<-- Add random string to click history (repeats unlikely)
      
      } 
      
    });
""")